In [1]:
from src.decode import read_source, read_test_source, parse_program, dump_file, parse_program_with_types
from src.funcs import *
from src.analyze import sorted_defs, get_unique_fns, get_unique_refs, get_all_unique_fns, is_token, inline, get_trivial_and_nontrivial_defs

In [2]:
defs = parse_program_with_types(read_source())

In [3]:
test_defs = parse_program_with_types(read_test_source())

In [4]:
test_defs

{ref::1: [1],
 ref::2: [2],
 ref::3: [3],
 ref::4: [<INC()>],
 ref::5: [<ADD()>],
 ref::6: [src.funcs.AP, <ADD()>, 2],
 ref::7: [src.funcs.AP, src.funcs.AP, <CONS()>, 4, 5],
 ref::8: [src.funcs.AP,
  src.funcs.AP,
  ref::5,
  src.funcs.AP,
  <CAR()>,
  ref::7,
  3],
 ref:galaxy: [ref::8]}

In [6]:
tree = AP()
tree.add_edge(AP())
tree.add_edge(ADD)
tree.add_edge(AP())
tree.add_edge(INC)
tree.add_edge(AP())
tree.add_edge(INC)
tree.add_edge(AP())
tree.add_edge(INC)
tree.add_edge(2)
tree.add_edge(4)
print(f'Tree complete: {tree.is_complete()}')
print(f'Tree: {tree}')
tree.resolve()

Tree complete: True
Tree: (AP (AP <ADD()> (AP <INC()> (AP <INC()> (AP <INC()> 2)))) 4)


9

In [9]:
AP(AP(CONS, 1), 2) == AP(AP(CONS, 1), 2)

True

In [4]:
get_all_unique_fns(defs)

set()

In [5]:
get_unique_refs(inline(defs, 'galaxy', 2))

{':1328', ':1336', ':1342'}

In [8]:
trivial_defs, nontrivial_defs = get_trivial_and_nontrivial_defs(defs, 2)
with open('galaxy.inlined_2.txt', 'w') as output:
    output.write('# Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in trivial_defs]))
    output.write('\n\n# Non-Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in nontrivial_defs]))

In [9]:
trivial_defs, nontrivial_defs = get_trivial_and_nontrivial_defs(defs, 3)
with open('galaxy.inlined_3.txt', 'w') as output:
    output.write('# Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in trivial_defs]))
    output.write('\n\n# Non-Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in nontrivial_defs]))

In [7]:
trivial_defs, nontrivial_defs = get_trivial_and_nontrivial_defs(defs, 10)
with open('galaxy.inlined_10.txt', 'w') as output:
    output.write('# Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in trivial_defs]))
    output.write('\n\n# Non-Trivial defs: \n')
    output.write('\n'.join([f"{token} = {lexems}" for token, lexems in nontrivial_defs]))